### 練習問題2.97
a. このアルゴリズムを reduce-terms という⼿続きとして実装せよ。<br />
⼿続きは n と d という⼆つの項リストを引数として取り、<br />
上で説明したアルゴリズムによって n と d を既約にした nn と dd のリストを返す。<br />
また、add-poly と同様の⼿続き reduce-poly を実装せよ。<br />
これは⼆つの poly が同じ変数を持つかどうかチェックし、<br />
もし同じであれば変数を取り去って問題を reduce-terms に渡し、<br />
reduce-terms に返された⼆つの項リストにその変数をつけ直す。<br />

b. reduce-terms と似たような⼿続きとして、<br />
元々の make-rat が整数に対して⾏っていたことと<br />
同じことを実⾏するものを次のように定義せよ。<br />

    (define (reduce-integers n d)
        (let ((g (gcd n d)))
            (list (/ n g) (/ d g))))

そして、reduce というジェネリック演算を定義せよ。<br />
これは、apply-generic を呼んで、reduce-poly(polynomial 型引数に対して) <br />
または reduce-integers(scheme-number 型引数に対して) にディスパッチする。<br />
こうすると、make-rat が分⼦と分⺟を組み合わせて有理数を作る前に reduce を呼ぶようにすることで、<br />
有理数算術演算パッケージが有理数を既約にするようにできる。<br />
これで、このシステムは整数と多項式のどちらの有理式も扱えるようになった。<br />
プログラムをテストするために、この拡張練習問題の最初に出てきた例を試してみよ。<br />

    (define p1 (make-polynomial 'x '((1 1) (0 1))))
    (define p2 (make-polynomial 'x '((3 1) (0 -1))))
    (define p3 (make-polynomial 'x '((1 1))))
    (define p4 (make-polynomial 'x '((2 1) (0 -1))))
    (define rf1 (make-rational p1 p2))
    (define rf2 (make-rational p3 p4))
    (add rf1 rf2)

正しく既約になっている正しい答えが返ってくるかどうかを
確認せよ。

In [1]:
(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))
(define get-coercion get)
(define put-coercion put)

; タグつきデータのコンストラクタ　練習問題2.78にあるsheme-numberは数値そのものとする
(define (attach-tag type-tag contents)
  (if (number? contents)
      contents
      (cons type-tag contents)))

; タグを返すセレクタ　練習問題2.78にあるsheme-numberは数値そのものとする
(define (type-tag datum)
  (cond ((number? datum) 'scheme-number)
        ((pair? datum) (car datum))
        (else (error "Bad tagged datum: TYPE-TAG" datum))))

; データを返すセレクタ　練習問題2.78にあるsheme-numberは数値そのものとする
(define (contents datum)
  (cond ((number? datum) datum)
        ((pair? datum) (cdr datum))
        (else (error "Bad tagged datum: CONTENTS" datum))))

;
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (if (not (eq? type1 type2))
                    (let ((t1->t2 (get-coercion type1 type2))
                          (t2->t1 (get-coercion type2 type1)))
                      (cond (t1->t2
                             ;(apply-generic op (t1->t2 a1) a2))
                             (apply-generic op (t1->t2 a1 a2) a2))
                            (t2->t1
                             ;(apply-generic op a1 (t2->t1 a2)))
                             (apply-generic op a1 (t2->t1 a2 a1)))
                            (else (error "No method for these types" (list op type-tags)))
                            )
                      )
                    (error "Same types" (list op type-tags))
                    )
                )
              (error "No method for these types" (list op type-tags)))))
      )
    )

; ジェネリック算術演算手続き
(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))
(define (=zero? x) (apply-generic '=zero? x))    ;追加手続き
(define (reverse-sign x) (apply-generic 'reverse-sign x))    ;追加手続き
(define (power x y) (apply-generic 'power x y))    ;べき数
(define (greatest-common-divisor x y) (apply-generic 'greatest-common-divisor x y))    ;追加手続き
(define (reduce x y) (apply-generic 'reduce x y))    ;追加手続き

; scheme-number
(define (install-scheme-number-package)
  (define (tag x) (attach-tag 'scheme-number x))
  (define (power-schme-number x y)
    (if (= y 0) 1
        (* x (power-schme-number x (- y 1)))
        )
    )
  (define (gcd a b)
    (if (= b 0) a
        (gcd b (remainder a b)))
    )
  (define (reduce-integers n d)
    (let ((g (gcd n d)))
      (list (/ n g) (/ d g))))  
  (put 'add '(scheme-number scheme-number) (lambda (x y) (tag (+ x y))))
  (put 'sub '(scheme-number scheme-number) (lambda (x y) (tag (- x y))))
  (put 'mul '(scheme-number scheme-number) (lambda (x y) (tag (* x y))))
  (put 'div '(scheme-number scheme-number) (lambda (x y) (tag (/ x y))))
  (put '=zero? '(scheme-number) (lambda (x) (= x 0)))    ;追加手続き
  (put 'reverse-sign '(scheme-number) (lambda (x) (* x -1)))    ;追加手続き
  (put 'power '(scheme-number scheme-number) (lambda (x y) (power-schme-number x y)))    ;べき数
  (put 'greatest-common-divisor '(scheme-number scheme-number) (lambda (x y) (tag (gcd x y))))  ; 追加手続き
  ; 追加手続き
  (put 'reduce '(scheme-number scheme-number)
       (lambda (x y)
         (let ((d (reduce-integers x y)))
           (list (tag (car d)) (tag (cadr d)))
         )
       )
    )
  (put 'make 'scheme-number (lambda (x) (tag x)))
'done)

(define (make-scheme-number n) ((get 'make 'scheme-number) n))

(install-scheme-number-package)

done

In [2]:
(define (install-rational-package)
  ;(define (gcd a b)
  ;  (if (= b 0) a
  ;      (gcd b (remainder a b)))
  ;  )
  ;; 内部⼿続き
  (define (numer x) (car x))
  (define (denom x) (cdr x))
  (define (make-rat n d)
    (let ((g (reduce n d)))
      ;(display g)
      ;(display "\n")
      (cons (car g) (cadr g))
      ;(cons n d)
    )
    )
  (define (add-rat x y) (make-rat (add (mul (numer x) (denom y))
                                     (mul (numer y) (denom x)))
                                  (mul (denom x) (denom y))))
  (define (sub-rat x y) (make-rat (sub (mul (numer x) (denom y))
                                     (mul (numer y) (denom x)))
                                  (mul (denom x) (denom y))))
  (define (mul-rat x y) (make-rat (mul (numer x) (numer y))
                                  (mul (denom x) (denom y))))
  (define (div-rat x y) (make-rat (mul (numer x) (denom y)) (mul (denom x) (numer y))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rational x))
  (put 'add '(rational rational) (lambda (x y) (tag (add-rat x y))))
  (put 'sub '(rational rational) (lambda (x y) (tag (sub-rat x y))))
  (put 'mul '(rational rational) (lambda (x y) (tag (mul-rat x y)))) 
  (put 'div '(rational rational) (lambda (x y) (tag (div-rat x y))))
  (put 'reverse-sign '(rational) (lambda (x) (mul (numer x) -1)))    ;追加手続き
  (put 'make 'rational (lambda (n d) (tag (make-rat n d))))
  'done)

(define (make-rational n d) ((get 'make 'rational) n d))

(install-rational-package)

done

In [3]:
; polynomialの実装
(define (install-polynomial-package)
  ; 内部⼿続き
  ; poly の表現
  (define (make-poly variable term-list) (cons variable term-list))
  (define (variable p) (car p))
  (define (term-list p) (cdr p))
  ; 2.3.2 節の same-variable? と variable? ⼿続き
  (define (same-variable? v1 v2) (and (variable? v1) (variable? v2) (eq? v1 v2)))
  (define (variable? x) (symbol? x))
  ; 項と項リストの表現
  ; 下記の adjoin-term . . . coeff ⼿続き
  (define (adjoin-term term term-list)
    (if (=zero? (coeff term))
        term-list
        (cons term term-list)))
  (define (the-empty-termlist) '())
  (define (first-term term-list) (car term-list))
  (define (rest-terms term-list) (cdr term-list))
  (define (empty-termlist? term-list) (null? term-list))
  (define (make-term order coeff) (list order coeff))
  (define (order term) (car term))
  (define (coeff term) (cadr term))
  (define (add-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1)
                   (add-terms (term-list p1) (term-list p2)))
        (error "Polys not in same var: ADD-POLY" (list p1 p2))))
  (define (mul-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1)
                   (mul-terms (term-list p1) (term-list p2)))
        (error "Polys not in same var: MUL-POLY" (list p1 p2))))  
  ; add-poly/mul-polyが使う⼿続き
  (define (add-terms L1 L2)
    (cond ((empty-termlist? L1) L2)
          ((empty-termlist? L2) L1)
          (else
           (let ((t1 (first-term L1))
                 (t2 (first-term L2)))
             (cond ((> (order t1) (order t2))
                    (adjoin-term
                     t1 (add-terms (rest-terms L1) L2)))
                   ((< (order t1) (order t2))
                    (adjoin-term
                     t2 (add-terms L1 (rest-terms L2))))
                   (else
                    (adjoin-term
                     (make-term (order t1)
                                (add (coeff t1) (coeff t2)))
                     (add-terms (rest-terms L1)
                                (rest-terms L2)))))))))
  (define (mul-terms L1 L2)
    (if (empty-termlist? L1)
        (the-empty-termlist)
        (add-terms (mul-term-by-all-terms (first-term L1) L2)
                   (mul-terms (rest-terms L1) L2))))
  (define (mul-term-by-all-terms t1 L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t2 (first-term L)))
          (adjoin-term
           (make-term (+ (order t1) (order t2))
                      (mul (coeff t1) (coeff t2)))
           (mul-term-by-all-terms t1 (rest-terms L))))))
  ; 符号反転
  (define (reverse-sign-proc p)
        (make-poly (variable p)
                   (reverse-sign-terms (term-list p)))    
    )
  (define (reverse-sign-terms L1)
    (if (empty-termlist? L1) (the-empty-termlist)
        (let ((t1 (first-term L1)))
          (adjoin-term
           (make-term (order t1) (reverse-sign (coeff t1)))
           (reverse-sign-terms (rest-terms L1))))))
  ; 減算
  (define (sub-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (add-poly p1 (reverse-sign-proc p2))
        (error "Polys not in same var: ADD-POLY" (list p1 p2))))
  (define (sub-terms t1 t2)
        (add-terms t1 (reverse-sign-terms t2))
    )
  (define (zero-terms L)
    (if (empty-termlist? L) #t
        (let ((t1 (first-term L)))
          (if (not (=zero? (coeff t1))) #f
              (zero-terms (rest-terms L))
          )
          )
        )
    )
  ; 強制型変換
  (define (scheme-number->polynomial n p)
    (tag (make-poly (variable (contents p)) (list (make-term 0 (contents n)))))
    )
  ; 除算
  (define (div-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (let ((ans (div-terms (term-list p1) (term-list p2))))
          ;ans
          (list (make-poly (variable p1) (car ans)) (make-poly (variable p2) (cadr ans))) 
            )
        ;(make-poly (variable p1)(div-terms (term-list p1) (term-list p2)))
        (error "Polys not in same var: DIV-POLY" (list p1 p2))
        )
    )
  (define (div-terms L1 L2)
    (if (empty-termlist? L1)
        (list (the-empty-termlist) (the-empty-termlist))
        (let ((t1 (first-term L1))
              (t2 (first-term L2)))
          (if (> (order t2) (order t1))
              (list (the-empty-termlist) L1)
              (let (
                    (new-c (div (coeff t1) (coeff t2)))
                    ;(new-c (div (coeff t1) (mul (coeff t2) (power (coeff t2) (add 1 (sub (order t1) (order t2)))))))
                    (new-o (sub (order t1) (order t2)))
                    ;(new-c2 (power (coeff t2) (add 1 (sub (order t1) (order t2)))))
                    )
                (let ((rest-of-result
                       ; 再帰的に残りを計算する
                       (div-terms (sub-terms L1 (mul-terms L2 (list (make-term new-o new-c)))) L2)
                  ))
                  ; 完全な結果を作る
                  (list (add-terms (list (make-term new-o new-c)) (car rest-of-result))
                        (cadr rest-of-result))
               ))))))
  ; 剰余
  (define (remainder-terms t1 t2)
    (let ((ans (div-terms t1 t2)))
          (cadr ans)
      )
    )  
  (define (pseudoremainder-terms L1 L2)
    (let ((ans (div-terms L1 L2))
          (t1 (first-term L1))
          (t2 (first-term L2))
          )
      (if (>= (add 1 (sub (order t1) (order t2))) 0)
          (let ((new-c (power (coeff t2) (add 1 (sub (order t1) (order t2))))))
            (mul-term-by-all-terms (make-term 0 new-c) (cadr ans)))
          (pseudoremainder-terms L2 L1)
        )
      )
    )
  (define (gcd-terms a b)
    (if (empty-termlist? b)
        (let ((coeffs (map coeff a)))
            (if (> (length coeffs) 1)
                (let ((d (accumulate greatest-common-divisor (car coeffs) (cdr coeffs)))
                      )
                  (mul-term-by-all-terms (make-term 0 (/ 1 d)) a)
                  )
                a
            )
          )
        (gcd-terms b (pseudoremainder-terms a b))))
  (define (gcd-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1) (gcd-terms (term-list p1) (term-list p2)))
        (error "Polys not in same var: GCD-POLY" (list p1 p2))
        )
    )
  ; 既約にする。簡略化
  (define (reduce-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (let ((ans (reduce-terms (term-list p1) (term-list p2))))
          (list (make-poly (variable p1) (car ans)) (make-poly (variable p2) (cadr ans))) 
            )
        (error "Polys not in same var: REDUCE-POLY" (list p1 p2))
        )
    )
  (define (reduce-terms L1 L2)
    (let ((d (gcd-terms L1 L2))
          )
      (if (empty-termlist? d) (list L1 L2)
          (list (car (div-terms L1 d)) (car (div-terms L2 d)))
          )
     )
    )
  ; システムのほかの部分とのインターフェイス
  (define (tag p) (attach-tag 'polynomial p))
  (put 'add '(polynomial polynomial) (lambda (p1 p2) (tag (add-poly p1 p2))))
  (put 'mul '(polynomial polynomial) (lambda (p1 p2) (tag (mul-poly p1 p2))))
  ;(put '=zero? '(polynomial) (lambda (p) (or (eq? p '()) (=zero? p))))  ; 追加手続き
  (put '=zero? '(polynomial) (lambda (p) (zero-terms (term-list p))))  ; 追加手続き
  (put 'reverse-sign '(polynomial) (lambda (p) (tag (reverse-sign-proc p))))  ; 追加手続き
  (put 'sub '(polynomial polynomial) (lambda (p1 p2) (tag (sub-poly p1 p2))))  ; 追加手続き
  ; 追加手続き
  (put 'div '(polynomial polynomial) (lambda (p1 p2)
                                       (let ((ans (div-poly p1 p2)))
                                         ;ans
                                         (list (tag (car ans)) (tag (cadr ans)))
                                         )
                                       )
       )
  ;(put 'remainder '(polynomial polynomial) (lambda (p1 p2) (tag (remainder-poly p1 p2))))  ; テスト用
  (put 'greatest-common-divisor '(polynomial polynomial) (lambda (p1 p2) (tag (gcd-poly p1 p2))))  ; 追加手続き
  ; 追加手続き
  (put 'reduce '(polynomial polynomial)
       (lambda (x y)
         (let ((ans (reduce-poly x y)))
           ;(display ans)
           ;(display "\n")
           (list (tag (car ans)) (tag (cadr ans)))
           )
       )
    )
  (put 'make 'polynomial (lambda (var terms) (tag (make-poly var terms))))
  (put-coercion  'scheme-number 'polynomial scheme-number->polynomial)
'done)

; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence)))
      )
  )

(define (make-polynomial var terms) ((get 'make 'polynomial) var terms))

(install-polynomial-package)

done

In [4]:
(define P1 (make-polynomial 'x '((2 1) (1 -2) (0 1))))
(define P2 (make-polynomial 'x '((2 11) (0 7))))
(define P3 (make-polynomial 'x '((1 13) (0 5))))
(define Q1 (mul P1 P2))
(define Q2 (mul P1 P3))

In [5]:
(greatest-common-divisor Q1 Q2)

(polynomial x (2 1) (1 -2) (0 1))

In [6]:
(reduce 32 24)

(4 3)

In [7]:
(reduce Q1 Q2)

((polynomial x (2 11) (0 7)) (polynomial x (1 13) (0 5)))

In [8]:
(make-rational 32 48)

(rational 2 . 3)

In [9]:
(define p1 (make-polynomial 'x '((1 1) (0 1))))
(define p2 (make-polynomial 'x '((3 1) (0 -1))))
(define p3 (make-polynomial 'x '((1 1))))
(define p4 (make-polynomial 'x '((2 1) (0 -1))))

In [10]:
(make-rational p3 p4)

(rational (polynomial x (1 -1)) polynomial x (2 -1) (0 1))

In [11]:
(make-rational p2 p1)

(rational (polynomial x (3 -1/2) (0 1/2)) polynomial x (1 -1/2) (0 -1/2))

In [12]:
(make-rational p1 p2)

(rational (polynomial x (1 -1/2) (0 -1/2)) polynomial x (3 -1/2) (0 1/2))

In [13]:
(define rf1 (make-rational p1 p2))

In [14]:
(define rf2 (make-rational p3 p4))

In [15]:
(add rf1 rf2)

(rational (polynomial x (3 1/2) (2 1) (1 3/2) (0 1/2)) polynomial x (4 1/2) (3 1/2) (1 -1/2) (0 -1/2))